In [1]:
import random
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':[1,11]}


In [2]:
class Card:
    def __init__(self, suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank + ' of ' + self.suit
    
class Deck():
    
    def __init__(self):
        self.all_cards = []
        self.cards_drawn = 0
        
        for suit in suits:
            for rank in ranks:
                created_card = Card(suit,rank)
                
                self.all_cards.append(created_card)
                
    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def check_all_cards_drawn(self):
        if self.cards_drawn > len(self.all_cards):
            self.shuffle()
        
    def deal_one(self):
        self.cards_drawn += 1
        self.check_all_cards_drawn()
        return self.all_cards.pop()

class Bank():
    
    def __init__(self):
        self.total_money = 500
    
    def bet(self, amount):
        self.total_money -= amount
        
    def gain(self, amount):
        self.total_money += amount
    
    def __str__(self):
        return f'You have a total of {self.total_money} euros'

class Player():
    
    def __init__(self, name):
        self.name = name
        self.cards = []
        self.bank = Bank()
    
    def add_cards(self, cards):
        if type(cards) != list:
            cards.value = self.check_ace(cards)
            self.cards.append(cards)
        else:
            for card in cards:  
                card.value = self.check_ace(card)
            self.cards.extend(cards)
        
    
    def check_ace(self, card):
        if (type(card.value) == list):
                while True:
                    selection = input("Please enter a number (1 or 11): ")
                    if selection == "1" or selection == "11":
                        break  # valid input, break out of the loop
                    else:
                        print("Invalid input. Please enter 1 or 11.")
                return int(selection)
        else:
            return card.value
    
    def sum_cards(self):
        total = 0
        for card in self.cards:
            if (type(card.value) == list):
                while True:
                    selection = input("Please enter a number (1 or 11): ")
                    if selection == "1" or selection == "11":
                        break  # valid input, break out of the loop
                    else:
                        print("Invalid input. Please enter 1 or 11.")
                card.value = int(selection)
                
            total += card.value
        return total
        
    def __str__(self):
        return f'{self.name} + has {len(self.cards)} cards'
    

In [3]:
game_on = True
game_deck = Deck()
game_deck.shuffle()

name = input("Player's name: ")
player = Player(name)
dealer = Player('Dealer')
bet = 0

def check_over21(p):
    if p.sum_cards() > 21:
        return True
    else:
        return False

print(f'\nWelcome {player.name} to the game!')
while game_on:
    
    bet = 0
    print(player.bank)
    #Check he has Money
    if player.bank.total_money > 0:
        while bet not in range(1, int(player.bank.total_money)+1):
            bet = int(input("Please Make a bet: "))
        player.bank.bet(bet)
        
    else:
        print('You have no money to keep playing. Thank you for coming!')
        game_on = False
        break
    
    
    ## START OF ROUND
    print("Let's begin the game!")
    
    # Draw 2 Cards for the Player
    player.add_cards( [game_deck.deal_one(), game_deck.deal_one()])
    print(f"\n{player.name} you have a {player.cards[0]} and a {player.cards[1]} and it makes {player.sum_cards()}")
    
    # Draw 2 Cards for the Dealer
    dealer.add_cards( [game_deck.deal_one(), game_deck.deal_one()])
    # Show 2 Cards of the Player and 1 of the Dealer to the Player
    print(f"\n{dealer.name} has a {dealer.cards[0]} and one hidden, the total shown makes {dealer.cards[0].value}")
    
    
    #Check BlackJack Player 
    if player.sum_cards() == 21:
        print(f"Blackjack!! you won the round, you won: {bet * 2.5}")
        player.bank.gain(bet * 2.5)
        
        
    else:
        ## PLAYER DECISION
        ## Let the player HIT or STAND
        decision = ''
        lost = False
        while True:
            decision = input('Do you want another card? (select typing with Y or N)')
            if decision.capitalize() in ['Y', 'N']:
                if decision.capitalize() == 'Y':
                    player.add_cards([game_deck.deal_one()])
                    print(f'You have drawn a {player.cards[-1]} of value {player.cards[-1].value}')

                    #CHECK IF LOSE
                    if check_over21(player) == True: 
                        print("You got over 21! You lost this round\n")
                        lost = True
                        break
                    else:
                        print(f"You now have a total of {player.sum_cards()}")


                elif decision.capitalize() == 'N':
                    break

        ## DEALER Turn
        if not lost:
            print(f"\n{dealer.name} has a {dealer.cards[0]} and a {dealer.cards[1]} and it makes {dealer.sum_cards()}")
            while dealer.sum_cards() < player.sum_cards():
                dealer.add_cards(game_deck.deal_one())
                print(f'Dealer has drawn a {dealer.cards[-1]} of value {dealer.cards[-1].value}')
                if check_over21(dealer) == True: 
                    print(f"Dealer got over 21! You won this round, you got: {bet * 2}")
                    player.bank.gain(bet * 2)
                    break
            else:
                print(f'Dealer has {dealer.sum_cards()}and has won this round! you lost {bet}')
            
            
        else:
            pass
        
    keep_playing = ''
    while keep_playing.capitalize() not in ['Y', 'N']:
        keep_playing = input(f'\nDo you want to keep playing? For the moment you have {player.bank.total_money} in the bank (select typing with Y or N)')
        if keep_playing.capitalize() == 'Y':
            break
        elif keep_playing.capitalize() == 'N':
            game_on = False
            break
    
    
    #Clear player and dealer Hands
    all_cards_played = []
    all_cards_played.extend(player.cards)
    all_cards_played.extend(dealer.cards)

    game_deck.all_cards.extend(all_cards_played)
    player.cards = []
    dealer.cards = []
    
print('\nThank you for Playing Black Jack! Hope to see you soon!')

        
            
    

    


Player's name: Pedro

Welcome Pedro to the game!
You have a total of 500 euros
Please Make a bet: 1
Let's begin the game!

Pedro you have a Four of Diamonds and a Seven of Diamonds and it makes 11

Dealer has a Jack of Hearts and one hidden, the total shown makes 10
Do you want another card? (select typing with Y or N)y
You have drawn a Five of Diamonds of value 5
You now have a total of 16
Do you want another card? (select typing with Y or N)n

Dealer has a Jack of Hearts and a Six of Spades and it makes 16
Dealer has 16and has won this round! you lost 1

Do you want to keep playing? For the momento you have 499 in the banky
You have a total of 499 euros
Please Make a bet: 500
Please Make a bet: 499
Let's begin the game!

Pedro you have a Six of Spades and a Jack of Hearts and it makes 16

Dealer has a Five of Diamonds and one hidden, the total shown makes 5
Do you want another card? (select typing with Y or N)n

Dealer has a Five of Diamonds and a Seven of Diamonds and it makes 12
De